In [12]:
import os
import polars as pl
import sgml, dproc

In [2]:
if not os.path.isfile('data/train.csv'):
    !kaggle competitions download -c playground-series-s4e7
    !mkdir data
    !unzip playground-series-s4e7.zip -d data
    !rm playground-series-s4e7.zip
    !kaggle datasets download -d annantkumarsingh/health-insurance-cross-sell-prediction-data
    !unzip health-insurance-cross-sell-prediction-data.zip
    !mv train.csv data/train_org.csv
    !mv test.csv data/test_org.csv
    !rm health-insurance-cross-sell-prediction-data.zip

100%|████████████████████████████████████████| 247M/247M [00:25<00:00, 9.73MB/s]
100%|████████████████████████████████████████| 247M/247M [00:25<00:00, 9.98MB/s]


Archive:  playground-series-s4e7.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


Archive:  health-insurance-cross-sell-prediction-data.zip
  inflating: test.csv                
  inflating: train.csv               
